In [ ]:
#  import some packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.metrics import roc_curve, auc
import numpy as np
from sklearn import metrics
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import LeaveOneOut

#  import data and specify patient diagnose and outcome

In [ ]:
#info = pd.read_csv('data/data_2states.txt', sep = '\t')
#features = pd.read_table('results/2state_ARI.txt',sep = ',')

info = pd.read_csv('data/data_2states.txt', sep = '\t')
features = pd.read_table('results/2state_ARI.txt',sep = ',')

data = pd.concat([info, features], axis = 1)
# drop unknown measures
data = data.drop(data[(data.Outcome_Prog == 3)].index)

In [ ]:
data

In [ ]:
ID=data['ID']

In [ ]:
dPLI=data['dPLI_ARI']

In [ ]:
Hub=data['Hub_ARI']

In [ ]:
Outcome=data['Outcome_Prog']
Diagnose=data['Outcome_Diag']
CRSR=data['CRSR']

In [ ]:
data= np.stack((ID, dPLI, Hub, Outcome,Diagnose,CRSR),axis=1)

I need to adapt the dataformat here into float and ineger (otherwise the numbers will be represented as a category)

In [ ]:
data=pd.DataFrame(data)
data.columns=['ID','dPLI','Hub','Outcome','Diagnose','CRSR']
data['dPLI']=data['dPLI'].astype(float)
data['Hub']=data['Hub'].astype(float)
data['Outcome']=data['Outcome'].astype(int)
data['Diagnose']=data['Diagnose'].astype(int)
data['CRSR']=data['CRSR'].astype(int)


This is how the data looks like

In [ ]:
data

In [ ]:

ax = sns.boxplot(x="Outcome", y="dPLI", data=data)
sns.swarmplot(x="Outcome", y="dPLI", data=data, color=".25", size = 6)
ax.set_title('dPLI_DRI')
ax.set_xticklabels(['Non-recovered','Recovered'])

In [ ]:

ax = sns.boxplot(x="Outcome", y="Hub", data=data)
sns.swarmplot(x="Outcome", y="Hub", data=data,color=".25",size = 6)
ax.set_title('Hub_DRI')
ax.set_xticklabels(['Non-recovered','Recovered'])

# Standardize the Data, show new data and plot

In [ ]:
X = data[['dPLI','Hub']]
scaler = preprocessing.StandardScaler()
X_stand = pd.DataFrame(scaler.fit_transform(X))
X_stand.columns=X.columns


In [ ]:
data_stand = data.copy()
data_stand['Hub']= X_stand['Hub'].copy()
data_stand['dPLI']= X_stand['dPLI'].copy()
data_stand['ARI']=data_stand['dPLI']+data_stand['Hub']
data_stand

# Perform Logistic Regression

In [ ]:
X_stand = data_stand[['dPLI','Hub']]
Y = data_stand['Outcome']

In [ ]:
c = 10
clf1 = LogisticRegression(random_state=0,penalty='l2',C = c).fit(X_stand, Y)
preds = clf1.predict(X_stand)
accuracy_score(Y,preds)


In [ ]:
# implement LOSO
loo = LeaveOneOut()
loo.get_n_splits(X_stand)

In [ ]:
LOSO_scores = []
Preds = []
for train_index, test_index in loo.split(X_stand):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X_stand.iloc[train_index], X_stand.iloc[test_index]
    y_train, y_test = Y[train_index], Y[test_index]
    #print(X_train, X_test, y_train, y_test)
    clf = LogisticRegression(random_state=0,penalty='l2', C = c).fit(X_train, y_train)
    pred = clf.predict(X_test)
    Preds.append(pred[0])
    acc = clf.score(X_test, y_test)
    print('LOSO Accuracy:  ', acc)
    LOSO_scores.append(acc)

In [ ]:
#Mean LOSO Score
print('Overall LOSO Accuracy:  ', np.mean(LOSO_scores))

In [ ]:
# Plot Model 

# Retrieve the model parameters.
b = clf1.intercept_[0]
w1, w2 = clf1.coef_.T
# Calculate the intercept and gradient of the decision boundary.
c = -b/w2
m = -w1/w2

# Plot the data and the classification with the decision boundary.
xmin, xmax = -1.3,3.5
ymin, ymax = -3, 2
#xmin, xmax = -0.2, 0.2
#ymin, ymax = 0, 2.5

xd = np.array([xmin, xmax])
yd = m*xd + c
plt.plot(xd, yd, 'k', lw=1, ls='--')
plt.fill_between(xd, yd, ymin, color='tab:orange', alpha=0.2)
plt.fill_between(xd, yd, ymax, color='tab:blue', alpha=0.2)

plt.scatter(X_stand['dPLI'][Y==1],X_stand['Hub'][Y==1], s = 30)
plt.scatter(X_stand['dPLI'][Y==0],X_stand['Hub'][Y==0], s = 30)
plt.xlim(xmin, xmax)
plt.ylim(ymin, ymax)
plt.ylabel('Standardized Hub Reconfiguration', fontsize = 12)
plt.xlabel('Standardized dPLI Reconfiguration', fontsize = 12)
plt.tick_params(axis='both', which='major', labelsize=12)

plt.show()

In [ ]:
#Confusion matrix, Accuracy, sensitivity and specificity
from sklearn.metrics import confusion_matrix

cm1 = confusion_matrix(Y,Preds)
print('Confusion Matrix : \n', cm1)

total1=sum(sum(cm1))
#####from confusion matrix calculate accuracy
accuracy1=(cm1[0,0]+cm1[1,1])/total1
print ('Accuracy : ', accuracy1)

sensitivity1 = cm1[0,0]/(cm1[0,0]+cm1[0,1])
print('Sensitivity : ', sensitivity1 )

specificity1 = cm1[1,1]/(cm1[1,0]+cm1[1,1])
print('Specificity : ', specificity1)

In [ ]:
# Plot Model differently 

fig = plt.figure()
# Plot the data and the classification with the decision boundary.
xmin, xmax = -1.5, 3.2
ymin, ymax = -2, 2.5

# define the x and y scale
x1grid = np.arange(xmin, xmax, 0.1)
x2grid = np.arange(ymin, ymax, 0.1)
# create all of the lines and rows of the grid
xx, yy = np.meshgrid(x1grid, x2grid)
# flatten each grid to a vector
r1, r2 = xx.flatten(), yy.flatten()
r1, r2 = r1.reshape((len(r1), 1)), r2.reshape((len(r2), 1))

# horizontal stack vectors to create x1,x2 input for the model
grid = np.hstack((r1,r2))
# define the model
model = LogisticRegression()
# fit the model
model.fit(X_stand, Y)
# make predictions for the grid
yhat = model.predict_proba(grid)
# keep just the probabilities for class 0
yhat = yhat[:, 0]
# reshape the predictions back into a grid
zz = yhat.reshape(xx.shape)*100
# plot the grid of x, y and z values as a surface
c = plt.contourf(xx, yy, zz, cmap='RdBu',alpha= 0.3)
# add a legend, called a color bar
plt.colorbar(c)
# create scatter plot for samples from each class

# create scatter of these samples
plt.scatter(X_stand['dPLI'][Y == 0], X_stand['Hub'][Y == 0], color = 'Blue',s = 50)
plt.scatter(X_stand['dPLI'][Y == 1], X_stand['Hub'][Y == 1], color = 'Red',s = 50)

# show the plot
plt.show()

In [ ]:
# GET ROC AUC
y_score = clf1.predict_proba(X_stand)[:,1]
# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()

fpr, tpr, _ = metrics.roc_curve(Y, y_score, pos_label=1)
roc_auc = auc(fpr, tpr)
plt.plot(fpr, tpr, color='darkorange',label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
plt.xlim([-0.05, 1.0])
plt.ylim([-0.05, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic for LOG REG')
plt.legend(loc="lower right")
